In [92]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import random

In [93]:
# Classe de base Mangeur
class User:
    def __init__(self, 
                 nom: str, 
                 prenom: str, 
                 age: int, 
                 sexe: str, 
                 user_id: int, 
                 classe_mangeur: str,
                 type_food: str, 
                ) -> None:
        """
        Initialize a user with their attributes
        """
        self.nom = nom
        self.prenom = prenom
        self.age = age
        self.sexe = sexe
        self.user_id = user_id
        self.classe_mangeur = classe_mangeur
        self.facteur_calories = 1.2 if sexe == 'homme' else 1.0
        self.heures_repas = {}
        self.intervalles_calories = {}
        self.type_food_file = type_food  # Remplacez par le chemin réel
        self.probabilites_df = pd.read_excel(self.type_food_file)
        self.aliments_consomme = []
        
    def generer_heures_connexion(self):
        heures_de_connexion = []
        for repas, heure in self.heures_repas.items():
            heure_reelle = datetime.strptime(heure, '%H:%M')
            variation = timedelta(minutes=random.randint(-60, 60))
            heure_variee = heure_reelle + variation
            heures_de_connexion.append((repas, heure_variee))
        return heures_de_connexion

    def choisir_types_aliments(self, repas):
        repas_col_avg = f'Meal_{repas}_avg'
        repas_col_std = f'Meal_{repas}_std'
        
        types = self.probabilites_df['Types']
        moyennes = self.probabilites_df[repas_col_avg]
        std_devs = self.probabilites_df[repas_col_std]

        probabilites = np.random.normal(moyennes, std_devs)
        probabilites = np.clip(probabilites, 0, None)  # S'assurer que les probabilités ne sont pas négatives

        total_probabilite = np.sum(probabilites)
        if total_probabilite > 0:
            probabilites = probabilites / total_probabilite  # Normaliser pour que la somme des probabilités soit 1

        types_choisis = np.random.choice(types, size=len(types), p=probabilites)
        return types_choisis

    def selectionner_aliments(self, aliments_df, types_choisis, repas, min_calories, max_calories):
        min_calories *= self.facteur_calories
        max_calories *= self.facteur_calories
        total_calories = 0
        aliments_selectionnes = []

        for type_aliment in types_choisis:
            aliments_du_type = aliments_df[aliments_df['Type'] == type_aliment]
            aliment_choisi = aliments_du_type.sample(n=1).iloc[0]
            aliment_choisi = aliment_choisi.to_dict()
            aliment_choisi['Repas'] = repas  # Ajouter l'information du repas
            total_calories += aliment_choisi['Valeur calorique']
            aliments_selectionnes.append(aliment_choisi)

            if total_calories >= min_calories:
                break

        # Si le total dépasse max_calories, retirer des aliments avec les plus faibles probabilités
        while total_calories > max_calories:
            aliments_selectionnes.sort(key=lambda x: self.probabilite_aliment(x['Type'], x['Repas']))
            aliment_a_retirer = aliments_selectionnes.pop(0)
            total_calories -= aliment_a_retirer['Valeur calorique']

        return aliments_selectionnes

    def probabilite_aliment(self, type_aliment, repas):
        repas_col_avg = f'Meal_{repas}_avg'
        moyenne = self.probabilites_df[self.probabilites_df['Types'] == type_aliment][repas_col_avg].values[0]
        return moyenne
        
    def simulate_daily_activity(self, business_date: date, aliments_df: pd.DataFrame):
        heures_de_connexion = self.generer_heures_connexion()
        self.aliments_consomme = []

        for repas, heure in heures_de_connexion:
            types_choisis = self.choisir_types_aliments(repas)
            aliments_selectionnes = self.selectionner_aliments(
                aliments_df, types_choisis, repas,
                self.intervalles_calories[repas][0],
                self.intervalles_calories[repas][1]
            )
            self.aliments_consomme.extend(aliments_selectionnes)
            print(f"{self.nom} s'est connecté à {heure.strftime('%Y-%m-%d %H:%M:%S')}")
            for aliment in aliments_selectionnes:
                print(f"{self.nom} a consommé {aliment['Aliment']} ({aliment['Valeur calorique']} calories) pour le repas {repas}")


    def get_daily_activity(self, business_date: date) -> dict:
        return {
            'date': business_date,
            'aliments_consomme': self.aliments_consomme
        }

# Création d'une instance utilisateur en fonction de la classe de mangeur
def create_user_instance(user_data: dict) -> User:
    classes_mangeurs = {
        'standard': MangeurStandard,
        # Ajoutez d'autres classes de mangeurs ici
    }
    classe_mangeur = user_data.pop('classe_mangeur')
    user_class = classes_mangeurs.get(classe_mangeur, User)
    return user_class(**user_data)

In [94]:
# Sous-classe pour les mangeurs standard
class MangeurStandard(User):
    def __init__(self, nom, prenom, age, sexe, user_id):
        super().__init__(nom, prenom, age, sexe, user_id, 'standard', 'standard_class.xlsx')
        self.heures_repas = {
            1: '08:00',  # petit_dejeuner
            2: '12:00',  # dejeuner
            3: '16:00',  # gouter
            4: '20:00'   # diner
        }
        self.intervalles_calories = {
            1: (300, 500),  # petit_dejeuner
            2: (600, 800),  # dejeuner
            3: (200, 300),  # gouter
            4: (500, 700)   # diner
        }

In [95]:
# Exemple d'utilisation
aliments_df = pd.read_excel(r"C:\Users\GUEGUEN\Desktop\WSApp\IM\DB\raw_food_data\aliments.XLSX")

# Créez un utilisateur de test de type MangeurStandard
test_user = MangeurStandard(nom="Test", prenom="User", age=25, sexe="homme", user_id=1)

# Simulez les activités quotidiennes pour une date spécifique
business_date = date.today()
test_user.simulate_daily_activity(business_date, aliments_df)

# Affichez les résultats de la simulation
print(f"Simulation des activités pour {test_user.nom} {test_user.prenom} le {business_date}:")
for aliment in test_user.aliments_consomme:
    print(f"{aliment['Aliment']} ({aliment['Valeur calorique']} calories) pour le repas {aliment['Repas']}")

Test s'est connecté à 1900-01-01 08:10:00
Test a consommé Tonic (114.0 calories) pour le repas 1
Test a consommé Mandarines, en conserve (92.0 calories) pour le repas 1
Test a consommé Café (2.0 calories) pour le repas 1
Test a consommé Jus orange et ananas (51.0 calories) pour le repas 1
Test a consommé Lait, écrémé (102.0 calories) pour le repas 1
Test s'est connecté à 1900-01-01 11:38:00
Test a consommé Bière légère, Budweiser (29.0 calories) pour le repas 2
Test a consommé Seiche, crue (79.0 calories) pour le repas 2
Test a consommé Sauce à pizza (136.0 calories) pour le repas 2
Test a consommé Eau (0.0 calories) pour le repas 2
Test a consommé Champignons de Paris (1.0 calories) pour le repas 2
Test a consommé Biscuit Chocolat Au Lait, Sondey (500.0 calories) pour le repas 2
Test s'est connecté à 1900-01-01 15:05:00
Test a consommé Lait, allégé en matières grasses (137.0 calories) pour le repas 3
Test s'est connecté à 1900-01-01 19:15:00
Test a consommé Petits pois & carottes, en 

In [96]:
# Classe MangeurStandard héritant de Mangeur
class MeatLover(Mangeur):
    def __init__(self, nom, sexe):
        heures_repas = {
            1: '08:00',  # petit_dejeuner
            2: '12:00',  # dejeuner
            3: '16:00',  # gouter
            4: '20:00'   # diner
        }
        intervalles_calories = {
            1: (400, 600),  # petit_dejeuner
            2: (700, 900),  # dejeuner
            3: (300, 500),  # gouter
            4: (600, 800)   # diner
        }
        type_food_file = "meat_lover_class.xlsx"
        super().__init__(nom, sexe, heures_repas, intervalles_calories, type_food_file)

In [54]:
meat_lover = MeatLover("Meat Lover", "homme")
meat_lover.connecter()
meat_lover.choisir_aliments(aliments_df)
meat_lover.afficher_aliments_consomme()

Meat Lover s'est connecté à 1900-01-01 08:19:00
Meat Lover s'est connecté à 1900-01-01 11:59:00
Meat Lover s'est connecté à 1900-01-01 15:40:00
Meat Lover s'est connecté à 1900-01-01 20:50:00
Meat Lover a consommé les aliments suivants :

Repas 1 :
 - Cupcake (196.0 calories)
 - Gésier de dinde, cuit (70.0 calories)
 - Beurre de graines de tournesol (99.0 calories)
 - Pomme, séchée (145.0 calories)
Calories pour le repas 1 = 510.0

Repas 2 :
 - Pamplemousse (53.0 calories)
 - Gésier de dinde, cuit (70.0 calories)
 - Palourdes, crues (17.0 calories)
 - Semoule (601.0 calories)
 - Pain pita (165.0 calories)
Calories pour le repas 2 = 906.0

Repas 3 :
 - Pain pita (165.0 calories)
 - Sardines à l'huile, en boîte (310.0 calories)
Calories pour le repas 3 = 475.0

Repas 4 :
 - Ormeau (89.0 calories)
 - Haricots noirs, crus (662.0 calories)
Calories pour le repas 4 = 751.0

Calories totales = 2642.0
